In [ ]:
# @title INSTALLING REQUIRED PACKAGES
!pip install nltk
!pip install python-docx
!pip install pytesseract
!pip install Pillow
!pip install langchain
!pip install transformers
!pip install sentence-transformers
!pip install pymupdf
!apt-get install -y tesseract-ocr
!pip install langdetect



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 925.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 783.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cud

In [ ]:
# @title IMPORTING PACKAGES
import nltk
import docx
from docx import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import pytesseract
from PIL import Image
from io import BytesIO
import os
import ipywidgets as widgets
from IPython.display import display
import time
from pathlib import Path
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import nltk
import numpy as np
from pathlib import Path
from langdetect import detect

# Download the punkt tokenizer data
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# @title UPLOAD FILE
# Create a file upload widget
uploader = widgets.FileUpload(
    accept='.docx',
    multiple=False,
    layout=widgets.Layout(width='200px', height='50px')
)

# Create a box to center the upload button
center_box = widgets.HBox([uploader], layout=widgets.Layout(justify_content='center'))

# Create a progress bar widget
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Uploading:',
    bar_style='',
    orientation='horizontal'
)

def simulate_progress(progress_widget, duration=2):
    steps = 100
    delay = duration / steps
    for i in range(steps):
        time.sleep(delay)
        progress_widget.value = i + 1

def on_upload_change(change):
    global file_path
    for filename, file_info in uploader.value.items():
        display(progress)
        simulate_progress(progress)
        file_path = os.path.join(os.getcwd(), filename)
        with open(file_path, 'wb') as f:
            f.write(file_info['content'])
        progress.value = 100
        print(f'File {filename} uploaded and saved to {file_path}')
        return file_path

uploader.observe(on_upload_change, names='value')
display(center_box)

IntProgress(value=0, description='Uploading:')

File Updated_Sample_Document_with_Various_Elements.docx uploaded and saved to /content/Updated_Sample_Document_with_Various_Elements.docx


In [ ]:
# @title DEFINE PATHS AND SETUP ENVIRONMENT
doc_path = file_path
doc = Document(doc_path)
os.chdir(os.path.dirname(doc_path))

temp_image_dir = "temp_images"
os.makedirs(temp_image_dir, exist_ok=True)

In [ ]:
# @title SETTING UP PARAMETERS
tokenized_data = list()
text_chunks = list()
max_length = 216  # @param {type: "integer"}
# Extract the first text paragraph to detect the language
first_text_sample = ""
for paragraph in doc.paragraphs:
    if paragraph.text.strip():
        first_text_sample = paragraph.text.strip()
        break

# Detect the language of the first text sample, default to English if no text detected
if first_text_sample:
    detected_language = detect(first_text_sample)
    print(f"Detected Language: {detected_language}")
else:
    detected_language = "en"  # Default to English if no text is detected
    print("No text detected, defaulting to English")

# Set up the appropriate tokenizer and embedder based on the detected language
if detected_language == "en":
    model_type = "English"
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    embedder = SentenceTransformer('bert-base-nli-mean-tokens')
elif detected_language == "tr":
    model_type = "Turkish"
    tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
    embedder = SentenceTransformer('dbmdz/distilbert-base-turkish-cased')
else:
    # Default to English if the language is neither English nor Turkish
    model_type = "English"
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    embedder = SentenceTransformer('bert-base-nli-mean-tokens')
    print(f"Unsupported language detected ({detected_language}), defaulting to English")

print(f"Using model type: {model_type}")

Detected Language: en


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Using model type: English


In [ ]:
# @title FUNCTION TO HANDLE IMAGES
def extract_images(doc_path):
    doc = docx.Document(doc_path)
    images = []
    for rel in doc.part.rels.values():
        if "image" in rel.target_ref:
            image = rel.target_part.blob
            images.append(image)
    return images

def extract_text_from_image(image_blob):
    image = Image.open(BytesIO(image_blob))
    text = pytesseract.image_to_string(image)
    return text


In [ ]:
# @title FUNCTION TO ITERATE THROUGH DOCUMENT ELEMENTS INCLUDING IMAGES
# FUNCTION TO ITERATE THROUGH DOCUMENT ELEMENTS INCLUDING IMAGES
def iter_block_items(doc):
    for block in doc.element.body:
        if block.tag.endswith('tbl'):
            yield docx.table.Table(block, doc)
        elif block.tag.endswith('p'):
            yield docx.text.paragraph.Paragraph(block, doc)
        elif block.tag.endswith('drawing') or block.tag.endswith('pict'):
            yield block

In [ ]:
#@title HELPER FUNCTION TO DETECT IMAGES INLINE WITH TEXT
def process_paragraph_with_images(paragraph):
    elements = []
    processed_images = set()

    for run in paragraph.runs:
        blips = run._element.xpath('.//a:blip')
        for blip in blips:
            embed_rel = blip.get('{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed')
            if embed_rel:
                rel = paragraph.part.rels[embed_rel]
                if rel.target_ref not in processed_images:
                    image_blob = rel.target_part.blob
                    ocr_text = extract_text_from_image(image_blob)
                    elements.append({'type': 'image', 'content': ocr_text})
                    processed_images.add(rel.target_ref)
        if run.text.strip():
            elements.append({'type': 'text', 'content': run.text.strip()})
    return elements

In [ ]:
# @title FUNCTION TO EXTRACT TEXT AND CHUNK CONTENT
# FUNCTION TO EXTRACT TEXT AND CHUNK CONTENT
def extract_text_and_chunk(doc):
    elements = []
    current_headline = None
    current_content = []

    for block in iter_block_items(doc):
        if isinstance(block, docx.text.paragraph.Paragraph):
            if block.style.name.startswith('Heading'):
                if current_headline:
                    elements.append({'type': 'section', 'headline': current_headline, 'content': ' '.join(current_content)})
                current_headline = block.text.strip()
                current_content = []
            else:
                # Process the paragraph and look for images
                paragraph_elements = process_paragraph_with_images(block)
                for element in paragraph_elements:
                    if element['type'] == 'text':
                        current_content.append(element['content'])
                    elif element['type'] == 'image':
                        if current_content:
                            elements.append({'type': 'section', 'headline': current_headline, 'content': ' '.join(current_content)})
                            current_content = []
                        elements.append(element)

        elif isinstance(block, docx.table.Table):
            if current_headline and current_content:
                elements.append({'type': 'section', 'headline': current_headline, 'content': ' '.join(current_content)})
                current_content = []
            table_content = []
            for row in block.rows:
                row_data = [cell.text.strip() for cell in row.cells]
                table_content.append(row_data)
            elements.append({'type': 'table', 'content': table_content})

    if current_headline and current_content:
        elements.append({'type': 'section', 'headline': current_headline, 'content': ' '.join(current_content)})

    return elements


In [ ]:
#@title # RECURSIVE FUNCTION TO SPLIT TEXT WITH SENTENCE OVERLAP
from nltk.tokenize import sent_tokenize

def split_text_recursive(text, max_length, overlap=2):
    sentences = sent_tokenize(text)

    def recursive_split(sentences, max_length, overlap):
        if not sentences:
            return []

        current_chunk = []
        while sentences and len(tokenizer.tokenize(' '.join(current_chunk + sentences[:1]))) <= max_length:
            current_chunk.append(sentences.pop(0))

        if len(sentences) <= overlap:
            return [current_chunk]

        # Recursive call, carrying over the last 'overlap' sentences
        return [current_chunk] + recursive_split(sentences[-(len(sentences) - overlap):], max_length, overlap)

    return [' '.join(chunk) for chunk in recursive_split(sentences, max_length, overlap)]

In [ ]:
# @title FUNCTION TO CONCATENATE TEXT AND IMAGE CHUNKS
def concatenate_text_and_images_with_overlap(elements):
    chunks = []

    for element in elements:
        if element['type'] == 'section':
            tokenized_content = element['content']
            text_chunks = split_text_recursive(tokenized_content, max_length=400)

            if text_chunks:
                chunks.append({'type': 'section', 'headline': element['headline'], 'content': text_chunks[0]})
                for text_chunk in text_chunks[1:]:
                    chunks.append({'type': 'section', 'content': text_chunk})

        elif element['type'] == 'image':
            chunks.append({'type': 'image', 'content': element['content']})

        elif element['type'] == 'table':
            content_json = json.dumps(element['content'], indent=4)
            chunks.append({'type': 'table', 'content': content_json})

    return chunks


In [ ]:
#@title FUNCTION TO EXTRACT ELEMENTS AND SAVE AS JSON
def extract_elements_to_json(doc):
    elements = []
    current_headline = None
    current_content = []

    for block in iter_block_items(doc):
        if isinstance(block, docx.text.paragraph.Paragraph):
            if block.style.name.startswith('Heading'):
                if current_headline and current_content:
                    elements.append({'type': 'section', 'headline': current_headline, 'content': ' '.join(current_content)})
                current_headline = block.text.strip()
                current_content = []
            else:
                # Process the paragraph and look for images
                paragraph_elements = process_paragraph_with_images(block)
                for element in paragraph_elements:
                    if element['type'] == 'text':
                        current_content.append(element['content'])
                    elif element['type'] == 'image':
                        if current_headline and current_content:
                            elements.append({'type': 'section', 'headline': current_headline, 'content': ' '.join(current_content)})
                            current_content = []
                        elements.append(element)

        elif isinstance(block, docx.table.Table):
            if current_headline and current_content:
                elements.append({'type': 'section', 'headline': current_headline, 'content': ' '.join(current_content)})
                current_content = []
            table_content = []
            for row in block.rows:
                row_data = [cell.text.strip() for cell in row.cells]
                table_content.append(row_data)
            elements.append({'type': 'table', 'content': table_content})

    if current_headline and current_content:
        elements.append({'type': 'section', 'headline': current_headline, 'content': ' '.join(current_content)})

    return elements

# MAIN SCRIPT EXECUTION FOR SAVING JSON
if __name__ == "__main__":
    # 1. Extract elements and save them to JSON
    elements = extract_elements_to_json(doc)
    json_output_path = Path('document_content.json')
    with open(json_output_path, 'w', encoding='utf-8') as json_file:
        json.dump(elements, json_file, ensure_ascii=False, indent=4)

    print(f"Document content saved as JSON to {json_output_path}")

    # 2. Load JSON content for further processing
    with open(json_output_path, 'r', encoding='utf-8') as json_file:
        loaded_elements = json.load(json_file)

Document content saved as JSON to document_content.json


In [ ]:
#@title Extract images and process them to get OCR texts and paths
elements = extract_text_and_chunk(doc)

# 2. Use the updated function for splitting and processing
final_chunks = concatenate_text_and_images_with_overlap(elements)

# 3. Display the final chunks
for i, chunk in enumerate(final_chunks):
    print(f"Chunk {i + 1} ({chunk['type'].capitalize()}):")
    if 'headline' in chunk:
        print(f"Headline: {chunk['headline']}")
    print(f"{chunk['content']}\n")



Chunk 1 (Section):
Headline: Introduction
This is an introductory paragraph in the sample document. It provides an overview of the document's content.

Chunk 2 (Section):
Headline: Main Content
Here is some main content. This content spans multiple sentences and paragraphs. It includes various elements such as text, tables, and images.

Chunk 3 (Image):
19:41

Search

< Owst

View your order

or Visit our store

als =)

Order summary

RED's White & Red
u Jersey x1 £12.00

Subtotal £12.00
Shipping £4.99
Taxes £0.00
Total £16.99 GBP

Customer information

Shipping address

Orhan Emir Ersanli

London

London SW 14


Chunk 4 (Table):
[
    [
        "Employee Name",
        "Department",
        "Salary",
        "Start Date",
        "Performance Rating"
    ],
    [
        "John Doe",
        "Sales",
        "$70,000",
        "01/15/2019",
        "A"
    ],
    [
        "Jane Smith",
        "Marketing",
        "$65,000",
        "03/22/2018",
        "B"
    ],
    [
        "Jim

In [ ]:
#@title Embedding
# Load the embedding model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Function to embed chunks and prepare them for saving
def embed_chunks(chunks):
    embedded_chunks = []

    for i, chunk in enumerate(chunks):
        content = chunk['content']
        # Embedding the content of each chunk
        if isinstance(content, str):
            embedding = model.encode(content).tolist()  # Convert numpy array to list
        elif isinstance(content, list):
            # If the content is a list (like a table), convert it to a string first
            content_str = ' '.join([str(row) for row in content])
            embedding = model.encode(content_str).tolist()  # Convert numpy array to list
        else:
            raise ValueError("Unsupported content type for embedding")

        # Store the embedding along with the chunk type
        embedded_chunk = {
            'type': chunk['type'],
            'embedding': embedding
        }
        embedded_chunks.append(embedded_chunk)

        # Print the embedding for display
        print(f"Chunk {i + 1} - Type: {embedded_chunk['type']}")
        print(f"Embedding: {np.array_str(np.array(embedded_chunk['embedding']), precision=3, suppress_small=True)}\n")

    return embedded_chunks


In [ ]:
#@title Function to save embeddings to a JSON file
def save_embeddings_to_json(embedded_chunks, output_path):
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(embedded_chunks, json_file, ensure_ascii=False, indent=4)
    print(f"Embeddings saved to {output_path}")

# MAIN SCRIPT EXECUTION FOR EMBEDDING AND SAVING
if __name__ == "__main__":
    # Assuming `final_chunks` is the list of chunks we processed earlier
    embedded_chunks = embed_chunks(final_chunks)

    # Define the output path for the JSON file
    embeddings_output_path = Path('embedded_chunks.json')

    # Save the embeddings to the JSON file
    save_embeddings_to_json(embedded_chunks, embeddings_output_path)

Chunk 1 - Type: section
Embedding: [-0.518  0.127  1.18   0.179  0.719 -0.688 -0.253  0.054  0.403 -0.304
  0.436  0.346  0.457 -0.088 -0.552  0.26  -0.381 -0.325  0.55  -0.563
 -0.189  0.121 -0.525 -0.094  0.788  0.243  0.336 -0.115 -0.503 -0.083
 -0.577 -1.409  0.48  -0.171 -0.367  0.217  0.814  0.854  0.244  0.158
 -0.877 -0.431  1.253  0.772 -1.251 -0.987 -0.172  0.499 -0.302 -0.458
 -0.823 -0.116  0.311  0.826 -0.881  0.412  0.117 -1.501 -0.608  0.221
 -0.198  0.357  0.64   0.171  0.705  0.599 -0.119 -0.353 -0.626  0.26
  0.203 -0.569 -1.099  1.14  -0.483 -0.136  0.521  0.697 -0.056 -0.026
 -0.461  0.259  0.393 -0.388  0.312  0.179  0.598 -0.032 -1.369  0.397
  0.585  0.093 -0.153 -0.574  0.511 -1.522 -0.039  0.228 -0.642  0.371
  0.339 -0.986  0.114 -0.054 -0.062  0.204  0.215 -0.689 -0.685  0.473
  0.538  0.077 -0.505  0.175 -0.467  0.424 -0.417 -0.162  0.022  0.887
 -0.295  0.415  0.123 -1.039  0.48  -0.187  1.043  1.371 -0.82   0.2
  0.764  0.239  1.275 -0.897 -0.676  0.175  0